## Download Dataset

In [1]:
!wget -nc https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/train.tsv
!wget -nc https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/dev.tsv

File ‘train.tsv’ already there; not retrieving.

File ‘dev.tsv’ already there; not retrieving.



In [2]:
!nvidia-smi

Thu May 18 15:22:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Packages

In [3]:
!pip install transformers==4.28.0
!pip install --upgrade keras
!pip install hanja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import torch
import numpy as np
import pandas as pd
import re
import hanja

from transformers import Trainer, TrainingArguments
from transformers import ElectraForSequenceClassification, ElectraTokenizer
from transformers import BertForSequenceClassification, BertTokenizerFast

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import seaborn as sn

## Load News Data & Process

In [5]:
train = pd.read_csv('train.tsv',sep='\t')
test = pd.read_csv('dev.tsv',sep='\t')

In [6]:
print('train data: {}, test data: {}'.format(len(train),len(test)))

train data: 7896, test data: 471


In [7]:
train.head(3)

comments  contain_gender_bias  \
0  (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...                False   
1  ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...                False   
2  ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...                False   

     bias  hate  
0  others  hate  
1    none  none  
2    none  hate

In [8]:
# We use only hate labels
train = train[['comments','hate']]
test = test[['comments','hate']]

### Preprocessing

In [9]:
def cleanse_sentence(sent): 
    
    # 문장 내 한자 제거
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sent) is not None:
        sent = hanja.translate(sent, 'substitution')
        
    # 문장 내 특수기호 제거
    sent = sent.replace('<p>', '\n').replace('</p>', '\n')
    sent = (r'\[[^\]]+\]|\([^\)]+\)|\<[^\>]+\>', '', str(sent))
    sent = re.sub("[.,!?||]", " ", str(sent))
    sent = re.sub(r'[^가-힣|\s]', '', str(sent))
    sent = re.sub(r'\s+', ' ', str(sent)) 
    
    return sent.rstrip()

In [10]:
train['comments'] = train['comments'].map(lambda x: cleanse_sentence(x))
test['comments'] = test['comments'].map(lambda x: cleanse_sentence(x))

<ipython-input-9-cb76ab9aecdc>:10: FutureWarning: Possible set union at position 5
  sent = re.sub("[.,!?||]", " ", str(sent))


In [11]:
train.head(3)

comments  hate
0    현재 호텔주인 심정 아 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네  hate
1   한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요  none
2   못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사...  hate

### Labeling

In [12]:
# Check values
train['hate'].value_counts()

none         3486
offensive    2499
hate         1911
Name: hate, dtype: int64

In [13]:
test['hate'].value_counts()

offensive    189
none         160
hate         122
Name: hate, dtype: int64

In [14]:
labels = train['hate'].value_counts().keys().tolist()

In [15]:
label_map = {v:i for i, v in enumerate(labels)}

In [16]:
label_map

{'none': 0, 'offensive': 1, 'hate': 2}

In [17]:
train['label'] = train['hate'].map(lambda x: label_map[x])
test['label'] = test['hate'].map(lambda x: label_map[x])

In [18]:
train.head(3)

comments  hate  label
0    현재 호텔주인 심정 아 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네  hate      2
1   한국적인 미인의 대표적인 분 너무나 곱고아름다운모습 그모습뒤의 슬픔을 미처 알지못했네요  none      0
2   못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사...  hate      2

## Load Model

In [19]:

# model = BertForSequenceClassification.from_pretrained('snunlp/KR-Medium', num_labels=3)
# tokenizer = BertTokenizerFast.from_pretrained('snunlp/KR-Medium')


model = ElectraForSequenceClassification.from_pretrained('snunlp/KR-ELECTRA-discriminator', num_labels=9)
tokenizer = ElectraTokenizer.from_pretrained('snunlp/KR-ELECTRA-discriminator')


Some weights of the model checkpoint at snunlp/KR-ELECTRA-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-ELECTRA-discriminator and are newly initialized: ['classifier.dense

### Check max length

In [20]:
def check_length(df):
  lens = []
  for sent in df['comments']:
    lens.append(len(tokenizer.tokenize(sent)))

  print('avg : {}, min : {}, max : {}'.format(sum(lens)/len(lens), min(lens),max(lens)))

In [21]:
check_length(train)

avg : 19.80483789260385, min : 1, max : 79


In [22]:
check_length(test)

avg : 20.0276008492569, min : 2, max : 66


## Build Dataset

In [23]:
#Make Dataset Class
class comments_data(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.label = torch.tensor(dataset['label']).long()
        features = tokenizer([str(x) for x in dataset.comments], padding=True, truncation=True,
                             max_length=128)
        self.input = torch.tensor(features['input_ids'])
        self.mask = torch.tensor(features['attention_mask'])
    
    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        return {'input_ids': self.input[index], 'attention_mask': self.mask[index], 'label':self.label[index]}

In [24]:
train_data = comments_data(train, tokenizer)
test_data = comments_data(test, tokenizer)

In [25]:
print('train data: {}, test data: {}'.format(len(train_data),len(test_data)))

train data: 7896, test data: 471


## Parameter setting

In [26]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [27]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
#Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15, 
    per_device_train_batch_size=32, # decrease batch size to avoid OOM error
    per_device_eval_batch_size=16,
    # warmup_steps=300,

    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    dataloader_pin_memory=False, 
    # weight_decay=0.01,
    # save_steps=1000,
    # logging_dir='./logs'
)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)

## Train

In [ ]:
#Train Model
trainer.train()

## Evaluate

In [ ]:
trainer.evaluate()

In [ ]:
# 나중을 위해 모델 저장
# trainer.save_model('kr-electra-hsp')